In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import to_date
from pyspark.sql.functions import format_number
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
spark.sparkContext.addFile(url)
home_sales = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.

home_sales.createOrReplaceTempView('houses')



In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query ="""
SELECT year(date) AS Year ,format_number(AVG(price),2) AS Avg_Price from houses
WHERE bedrooms==4
GROUP BY year(date)
ORDER BY year(date) DESC
"""

spark.sql(query).show()



+----+----------+
|Year| Avg_Price|
+----+----------+
|2022|296,363.88|
|2021|301,819.44|
|2020|298,353.78|
|2019|300,263.70|
+----+----------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query ="""
SELECT year(date) AS Year ,format_number(AVG(price),2) AS Avg_Price from houses
WHERE
bathrooms==2 AND
bedrooms==3
GROUP BY year(date)
ORDER BY year(date) DESC
"""

spark.sql(query).show()


+----+----------+
|Year| Avg_Price|
+----+----------+
|2022|286,490.11|
|2021|286,272.41|
|2020|288,523.85|
|2019|285,784.82|
+----+----------+



In [8]:
home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query ="""
SELECT year(date) AS Year ,format_number(AVG(price),2) AS Avg_Price from houses
WHERE
bathrooms==3 AND
bedrooms==3 AND
floors==2 AND
sqft_living>= 2000
GROUP BY year(date)
ORDER BY year(date) DESC
"""

spark.sql(query).show()


+----+----------+
|Year| Avg_Price|
+----+----------+
|2022|290,242.99|
|2021|296,330.96|
|2020|292,289.09|
|2019|289,859.14|
+----+----------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query ="""
SELECT view AS Views ,format_number(AVG(price),2) AS Avg_Price from houses
WHERE
price>=350000
GROUP BY view
ORDER BY view DESC
"""

spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----+------------+
|Views|   Avg_Price|
+-----+------------+
|  100|1,026,669.50|
|   99|1,061,201.42|
|   98|1,053,739.33|
|   97|1,129,040.15|
|   96|1,017,815.92|
|   95|1,054,325.60|
|   94|1,033,536.20|
|   93|1,026,006.06|
|   92|  970,402.55|
|   91|1,137,372.73|
|   90|1,062,654.16|
|   89|1,107,839.15|
|   88|1,031,719.35|
|   87|1,072,285.20|
|   86|1,070,444.25|
|   85|1,056,336.74|
|   84|1,117,233.13|
|   83|1,033,965.93|
|   82|1,063,498.00|
|   81|1,053,472.79|
+-----+------------+
only showing top 20 rows

--- 0.8197021484375 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table houses")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('houses')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query ="""
SELECT view AS Views ,format_number(AVG(price),2) AS Avg_Price from houses
WHERE
price>=350000
GROUP BY view
ORDER BY view DESC
"""

spark.sql(query).show()



print("--- %s seconds ---" % (time.time() - start_time))


+-----+------------+
|Views|   Avg_Price|
+-----+------------+
|  100|1,026,669.50|
|   99|1,061,201.42|
|   98|1,053,739.33|
|   97|1,129,040.15|
|   96|1,017,815.92|
|   95|1,054,325.60|
|   94|1,033,536.20|
|   93|1,026,006.06|
|   92|  970,402.55|
|   91|1,137,372.73|
|   90|1,062,654.16|
|   89|1,107,839.15|
|   88|1,031,719.35|
|   87|1,072,285.20|
|   86|1,070,444.25|
|   85|1,056,336.74|
|   84|1,117,233.13|
|   83|1,033,965.93|
|   82|1,063,498.00|
|   81|1,053,472.79|
+-----+------------+
only showing top 20 rows

--- 0.5251893997192383 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales.write.partitionBy("date_built").mode("overwrite").parquet("date_partitioned")

In [15]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('date_partitioned')

In [16]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('houses_p')

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query ="""
SELECT view AS Views ,format_number(AVG(price),2) AS Avg_Price from houses_p
WHERE
price>=350000
GROUP BY view
ORDER BY view DESC
"""



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00014209747314453125 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table houses")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("houses")

False